##### The Odds API Setup
- 500 request limit on Free tier
- Important note: the lines are delayed by several minutes, meaning I may need another scraping tool

In [43]:
import requests
from pandas import json_normalize

API_KEY = "fc35e20b0a8d983f77fc2fa9257a35c7"
SPORTS_URL = f"https://api.the-odds-api.com/v4/sports?apiKey={API_KEY}"
ODDS_URL = f"https://api.the-odds-api.com/v4/sports/baseball_mlb_preseason/odds/?apiKey={API_KEY}&regions=us,spreads&oddsFormat=american"


response = requests.get(SPORTS_URL)
sportsbook_data = response.json()
sports = json_normalize(sportsbook_data)
sports[sports["group"] == "Baseball"]

,key,group,title,description,active,has_outrights
4,baseball_mlb_preseason,Baseball,MLB Preseason,Major League Baseball,True,False
5,baseball_mlb_world_series_winner,Baseball,MLB World Series Winner,World Series Winner 2024,True,True
6,baseball_ncaa,Baseball,NCAA Baseball,US College Baseball,True,False


#### Testing theoddsapihelper.py

In [53]:
from theoddsapihelper import TheOddsAPIHelper

oddsapi = TheOddsAPIHelper()
full_odds = oddsapi.get_odds_table()
full_odds.to_csv("test_selected_sports_results.csv")

print(oddsapi.requests_made)

In [52]:
oddsapi.requests_made

'68'

#### Initial Data Ingestion
- Will code most of this in pandas but I need to see if a spark/polars dataframe would work better
long term

In [50]:
import pandas as pd

sportsbook = pd.DataFrame(sportsbook_data)

In [18]:
import polars as pl

pl_sportsbook = pl.DataFrame(sportsbook_data)

In [19]:
# Sample pyspark code

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# JSON data
api_response = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": 2, "name": "Bob", "age": 30},
    {"id": 3, "name": "Charlie", "age": 22},
]

# Define the schema for the DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
])

# Create a PySpark DataFrame
df = spark.createDataFrame(api_response, schema=schema)

# Show the DataFrame
df.show()


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 22|
+---+-------+---+



#### Preview sample dataset, top 40 results for NBA

In [105]:
result_df.head(40)

,outcome_name,outcome_price,outcome_point,id,bookmaker_key,market_key,market_last_update,bookmaker_title,bookmaker_last_update,sport_key,sport_title,commence_time,home_team,away_team
0,Los Angeles Lakers,-145,NaN,589eed81cc5543b6147d9b6b1c973c08,betmgm,h2h,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
1,Sacramento Kings,120,NaN,589eed81cc5543b6147d9b6b1c973c08,betmgm,h2h,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
2,Los Angeles Lakers,-110,-2.5,589eed81cc5543b6147d9b6b1c973c08,betmgm,spreads,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
3,Sacramento Kings,-110,2.5,589eed81cc5543b6147d9b6b1c973c08,betmgm,spreads,2024-03-06T04:56:19Z,BetMGM,2024-03-06T04:56:19Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
4,Los Angeles Lakers,-110,-2.5,589eed81cc5543b6147d9b6b1c973c08,betonlineag,spreads,2024-03-06T04:56:38Z,BetOnline.ag,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
5,Sacramento Kings,-110,2.5,589eed81cc5543b6147d9b6b1c973c08,betonlineag,spreads,2024-03-06T04:56:38Z,BetOnline.ag,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
6,Los Angeles Lakers,-137,NaN,589eed81cc5543b6147d9b6b1c973c08,betrivers,h2h,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
7,Sacramento Kings,116,NaN,589eed81cc5543b6147d9b6b1c973c08,betrivers,h2h,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
8,Los Angeles Lakers,-109,-2.5,589eed81cc5543b6147d9b6b1c973c08,betrivers,spreads,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
9,Sacramento Kings,-112,2.5,589eed81cc5543b6147d9b6b1c973c08,betrivers,spreads,2024-03-06T04:56:38Z,BetRivers,2024-03-06T04:56:38Z,basketball_nba,NBA,2024-03-07T03:40:00Z,Los Angeles Lakers,Sacramento Kings
